In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config

from config import user
from config import admin

In [2]:
netflix_file = "Resources/Netflix_Movies.csv"
netflix_df = pd.read_csv(netflix_file)
netflix_df.head(2)

,Unnamed: 0,title,director,cast,country,release_year
0,0,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2020
1,1,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,2021


In [3]:
c_netflix_df = netflix_df.drop(columns=["Unnamed: 0"])

In [4]:
c_netflix_df = c_netflix_df.rename(columns={"cast": "movie_cast"})

In [5]:
c_netflix_df['id'] = c_netflix_df.index
c_netflix_df.head()

,title,director,movie_cast,country,release_year,id
0,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2020,0
1,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,2021,1
2,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",1993,2
3,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021,3
4,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021,4


In [6]:
protocol = 'postgresql'
username = user
password = admin
host = 'localhost'
port = 5432
database_name = 'movies'

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
        
engine = create_engine(rds_connection_string)

In [7]:
inspector = inspect(engine)

inspector.get_table_names()

['netflix', 'disney_plus']

In [8]:
c_netflix_df.to_sql(name='netflix', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "netflix_pkey"
DETAIL:  Key (title)=(Dick Johnson Is Dead) already exists.

[SQL: INSERT INTO netflix (title, director, movie_cast, country, release_year, id) VALUES (%(title)s, %(director)s, %(movie_cast)s, %(country)s, %(release_year)s, %(id)s)]
[parameters: ({'title': 'Dick Johnson Is Dead', 'director': 'Kirsten Johnson', 'movie_cast': None, 'country': 'United States', 'release_year': 2020, 'id': 0}, {'title': 'My Little Pony: A New Generation', 'director': 'Robert Cullen, José Luis Ucha', 'movie_cast': 'Vanessa Hudgens, Kimiko Glenn, James Marsden, Sofia Carson, Liza Koshy, Ken Jeong, Elizabeth Perkins, Jane Krakowski, Michael McKean, Phil LaMarr', 'country': None, 'release_year': 2021, 'id': 1}, {'title': 'Sankofa', 'director': 'Haile Gerima', 'movie_cast': 'Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri', 'country': 'Uni

In [9]:
pd.read_sql_query('select * from netflix', con=engine).head()

,id,title,director,movie_cast,country,release_year
0,0,Dick Johnson Is Dead,Kirsten Johnson,None,United States,2020
1,1,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",None,2021
2,2,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",1993
3,3,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021
4,4,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021
